# Entrenamiento iterativo

In [1]:
import os
import re
import json
import joblib
import rasterio
import numpy as np
import pandas as pd
from glob import glob
from pyproj import CRS
from copy import deepcopy
from sqlite3 import connect
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from utilities import *

## Conjunto de entrenamiento

In [2]:
# cambiar según corresponda
# train_sqlite_files debe contener los .sqlite generados a partir de la verdad de campo
train_sqlite_files = glob('../data/selection_verdad_campo/*.sqlite')

train_data = pd.DataFrame()

for sf in train_sqlite_files:
    file_name = os.path.basename(sf)
    tile = re.search(r'\d+',file_name).group()
    cnx = connect(sf)
    df = pd.read_sql_query("SELECT * FROM output", cnx)
    df['tile_file'] = tile
    train_data = pd.concat([train_data, df], ignore_index=True)

In [3]:
train_data.shape

(466, 35)

In [4]:
train_data.head()

,ogc_fid,GEOMETRY,in1,id,cultivo,originfid,band_0,band_1,band_2,band_3,...,band_19,band_20,band_21,band_22,band_23,band_24,band_25,band_26,band_27,tile_file
0,1,b'\x01\x01\x00\x00\x00 \xf1\x0b\xeaL\xd5O\xc0\...,014084,1,SOJA,14,0.48176,0.765948,0.3583,0.788596,...,5.113691,1.8320,0.387479,0.855175,0.593264,0.863072,3.181,0.115103,0.875580,00000
1,2,b'\x01\x01\x00\x00\x00V\x8b\x9d/\x0b\xabO\xc0T...,014084,1,SOJA,34,0.47336,0.762060,0.9424,0.782061,...,4.478749,3.1410,0.319588,0.843794,0.484375,0.839304,2.263,0.636738,0.845511,00000
2,3,b'\x01\x01\x00\x00\x00M\xa8\xdf4\xe5\xa4O\xc0\...,014084,1,SOJA,35,0.49016,0.788664,0.7118,0.799592,...,4.509433,2.1070,0.323782,0.844182,0.515982,0.833669,3.636,0.382938,0.847742,00000
3,4,b'\x01\x01\x00\x00\x00\xe4\x02\xab\x1d\xb0\xd7...,014084,1,SOJA,38,0.52476,0.826668,0.8436,0.832025,...,5.455678,1.8985,0.399654,0.866689,0.607895,0.876895,1.886,0.471242,0.888227,00000
4,5,b'\x01\x01\x00\x00\x00\xa7\xe5\x99\xec^\xd4O\x...,014084,1,SOJA,39,0.53340,0.847746,0.8716,0.840895,...,5.325085,2.2785,0.423227,0.853557,0.647971,0.868240,2.473,0.543610,0.883840,00000


## Conjunto de predicción

In [5]:
# cambiar según corresponda
# pred_sqlite_files debe contener los .sqlite generados a partir de la máscara mask_agri_aoi
pred_sqlite_files = glob('../data/selection_mask_agri/*.sqlite')

pred_data = pd.DataFrame()

for sf in pred_sqlite_files:
    file_name = os.path.basename(sf)
    tile = re.search(r'\d+',file_name).group()
    cnx = connect(sf)
    df = pd.read_sql_query("SELECT * FROM output", cnx)
    df['tile_file'] = tile
    pred_data = pd.concat([pred_data, df], ignore_index=True)

In [6]:
pred_data.shape

(2000, 34)

In [7]:
pred_data.head()

,ogc_fid,GEOMETRY,fid,dn,originfid,band_0,band_1,band_2,band_3,band_4,...,band_19,band_20,band_21,band_22,band_23,band_24,band_25,band_26,band_27,tile_file
0,1,b'\x01\x01\x00\x00\x00\x86\xe9\xb0?<\xe7O\xc0\...,out.119,1,26,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,00000
1,2,b'\x01\x01\x00\x00\x00\x0c1\x99\xc1\xf1\x00P\x...,out.6054,1,2252,0.52864,0.833700,0.81190,0.836442,0.798558,...,5.209053,3.02225,0.361667,0.867252,0.573316,0.865887,3.4590,0.433533,0.879380,00000
2,3,b'\x01\x01\x00\x00\x00\xb0\x80|\xd9!\xe8O\xc0A...,out.7360,1,2855,0.09364,0.286680,0.32240,0.355726,0.227484,...,1.220818,1.03600,-0.009247,0.573864,-0.013527,0.380513,2.0955,0.102407,0.360268,00000
3,4,"b""\x01\x01\x00\x00\x00u:2F\x15\xffO\xc0\xce\xf...",out.8476,1,3326,0.09192,0.288402,0.38635,0.363033,0.226837,...,1.182418,0.74950,-0.043204,0.585802,-0.061712,0.380720,1.3330,0.147536,0.342803,00000
4,5,b'\x01\x01\x00\x00\x00\xcf\xfd\x89\xfci\xf8O\x...,out.8485,1,3328,0.16676,0.380328,0.25490,0.430185,0.317554,...,1.767914,0.38800,0.161189,0.598485,0.235429,0.517596,2.5700,0.063620,0.515918,00000


## Agregado de departamentos

In [8]:
# cambiar según corresponda
# deptos_sqlite_files debe contener los .sqlite generados a partir de los departamentos
deptos_sqlite_files = glob('../data/selection_departamentos_features/selection_departamentos/*.sqlite')

deptos_data = pd.DataFrame()

for sf in deptos_sqlite_files:
    file_name = os.path.basename(sf)
    tile = re.search(r'\d+',file_name).group()
    cnx = connect(sf)
    df = pd.read_sql_query("SELECT * FROM output", cnx)
    df['tile_file'] = tile
    deptos_data = pd.concat([deptos_data, df], ignore_index=True)

In [9]:
train_data = train_data.merge(deptos_data[['ogc_fid','nombre','tile_file']], how='left', on=['ogc_fid','tile_file'])
pred_data = pred_data.merge(deptos_data[['ogc_fid','nombre','tile_file']], how='left', on=['ogc_fid','tile_file'])

In [10]:
# train_data está dentro de pred_data
# saco esas filas de pred_data
# así no predecimos con lo mismo con lo que entrenamos

merged_data = pred_data.merge(train_data[['ogc_fid','nombre','tile_file','id','cultivo']], how='left', on=['ogc_fid','tile_file','nombre'], indicator=True)
train_data = merged_data[merged_data._merge=='both'].drop(columns=['_merge'])
pred_data = merged_data[merged_data._merge=='left_only'].drop(columns=['_merge'])

In [11]:
train_data.shape

(466, 37)

In [12]:
pred_data.shape

(1534, 37)

## Entrenamiento del modelo

In [13]:
map_id2cultivo = dict((
    train_data[['id','cultivo']]
    .drop_duplicates()
    .assign(id=lambda x: x.id.astype('int'))
    .itertuples(index=False, name=None))
)
map_id2cultivo

{1: 'SOJA',
 2: 'MAIZ',
 20: 'CAMPONATUR',
 3: 'MAIZ',
 5: 'GIRASOL',
 10: 'ALFALFA',
 4: 'SOJA'}

In [14]:
le = LabelEncoder()
le.fit(train_data.id)

LabelEncoder()

In [15]:
map_le2id = dict(zip(le.transform(le.classes_), list(map(int,le.classes_))))

map_le2id

{0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 10, 6: 20}

In [16]:
train_data['id_le'] = le.transform(train_data.id)

train_data.head()

,ogc_fid,GEOMETRY,fid,dn,originfid,band_0,band_1,band_2,band_3,band_4,...,band_23,band_24,band_25,band_26,band_27,tile_file,nombre,id,cultivo,id_le
0,1,b'\x01\x01\x00\x00\x00\x86\xe9\xb0?<\xe7O\xc0\...,out.119,1,26,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,00000,GENERAL ROCA,1.0,SOJA,0
1,2,b'\x01\x01\x00\x00\x00\x0c1\x99\xc1\xf1\x00P\x...,out.6054,1,2252,0.52864,0.833700,0.81190,0.836442,0.798558,...,0.573316,0.865887,3.4590,0.433533,0.879380,00000,GENERAL ROCA,1.0,SOJA,0
2,3,b'\x01\x01\x00\x00\x00\xb0\x80|\xd9!\xe8O\xc0A...,out.7360,1,2855,0.09364,0.286680,0.32240,0.355726,0.227484,...,-0.013527,0.380513,2.0955,0.102407,0.360268,00000,GENERAL ROCA,1.0,SOJA,0
3,4,"b""\x01\x01\x00\x00\x00u:2F\x15\xffO\xc0\xce\xf...",out.8476,1,3326,0.09192,0.288402,0.38635,0.363033,0.226837,...,-0.061712,0.380720,1.3330,0.147536,0.342803,00000,GENERAL ROCA,1.0,SOJA,0
4,5,b'\x01\x01\x00\x00\x00\xcf\xfd\x89\xfci\xf8O\x...,out.8485,1,3328,0.16676,0.380328,0.25490,0.430185,0.317554,...,0.235429,0.517596,2.5700,0.063620,0.515918,00000,GENERAL ROCA,1.0,SOJA,0


In [17]:
with open('../model/randomforest_parameters.json','r') as f:
    parameters = json.load(f)

In [20]:
!ls

Untitled.ipynb		       randomforest_iter_01.ipynb
__pycache__		       randomforest_iteration2.ipynb
convert_csv2tif.ipynb	       randomforest_iterations.ipynb
eda_results.ipynb	       randomforest_iterations3.ipynb
eval_train.ipynb	       utilities.py
randomforest_eval_train.ipynb


In [18]:
thresholds= [0.3,0.4,0.56,0.6, 0.65,0.7,0.8]

In [19]:
for threshold in thresholds:
    
    iter_train_data = deepcopy(train_data)
    iter_pred_data = deepcopy(pred_data)
    
    print(f'+++++ PREDICCIONES PARA THRESHOLD {threshold}')
    
    threshold_folder = os.path.join('..','model',f'threshhold_{threshold}')
    os.makedirs(threshold_folder, exist_ok=True)
    
    i = 0
    while True:
        
        # arma carpeta para el output (i aumenta con las iteraciones)
        n_iter = '{0:03d}'.format(i)
        output_folder = os.path.join(threshold_folder,f'randomforest_iterations_{n_iter}')
        os.makedirs(output_folder, exist_ok=True)
        
        # toma los datasets
        columns = iter_train_data.filter(regex='band_').columns.to_list()
        X_train = iter_train_data.filter(regex='band_').fillna(-99999).to_numpy()
        y_train = iter_train_data.id_le.to_numpy()
        X_pred = iter_pred_data.filter(regex='band_').fillna(-99999).to_numpy()
        
        # instancia y entrena el modelo
        model = RandomForestClassifier(**parameters)
        model.fit(X_train, y_train)
        output_model_file = os.path.join(output_folder, f'model_{n_iter}.joblib')
        _ = joblib.dump(model, output_model_file)
        
        # predice
        probas = model.predict_proba(X_pred)
        output_proba_file = os.path.join(output_folder, f'probas_{n_iter}.npy')
        np.save(output_proba_file, probas)
        predictions = iter_pred_data.assign(pred_class=probas.argmax(axis=1), pred_score=probas.max(axis=1))
        
        # separa entre nuevo train y nuevo pred
        add_to_train = predictions.query(f'pred_score >= {threshold}').copy()
        continue_pred = predictions.query(f'pred_score < {threshold}').copy()
        train_data_len, add_to_train_len , continue_pred_len = iter_train_data.shape[0], add_to_train.shape[0] , continue_pred.shape[0]
        output_pixels_file = os.path.join(output_folder, f'pixels_{n_iter}.csv')
        (
            pd.DataFrame(
                [
                    [f'De entrenamiento', train_data_len],
                    [f'Con proba>={threshold}', add_to_train_len],
                    [f'Con proba<{threshold}', continue_pred_len]
                ],
                columns=['Pyxels_type','Pixels']
            )
            .to_csv(output_pixels_file, index=False)
        )
        
        output_deptos_file = os.path.join(output_folder, f'pred_deptos_{n_iter}.csv')
        (
            pd.DataFrame(
                [
                    [f'De entrenamiento', train_data_len],
                    [f'Con proba>={threshold}', add_to_train_len],
                    [f'Con proba<{threshold}', continue_pred_len]
                ],
                columns=['Pyxels_type','Pixels']
            )
            .to_csv(output_pixels_file, index=False)
        )
        
        # pasa predicción a las columna id (target)
        # y lo agrega al train original
        add_to_train['id_le'] = add_to_train['pred_class']
        add_to_train['id'] = add_to_train.id_le.apply(lambda x: map_le2id.get(x))
        add_to_train['cultivo'] = add_to_train.id.apply(lambda x: map_id2cultivo.get(x))

        iter_deptos_prediction = os.path.join(output_folder,f'randomforest__deptos_prediction_{n_iter}.csv')
        (
            add_to_train
            .groupby(['cultivo','nombre'], as_index=False)
            .size()
            .rename(columns={'size':'pixels'})
            .assign(ha=lambda x: x.pixels*0.04)
            .to_csv(iter_deptos_prediction, index=False)
        )
        iter_train_data =iter_train_data.append(add_to_train, ignore_index=True)
        iter_pred_data = continue_pred
        
        # imprime información
        print('''\n*** ITERACIÓN #{0:03d}
        - Modelo guardado en {1}
        - Probabilidades guardadas en {2}
        - Pixeles de entrenamiento: {3}
        - Pixeles con proba>={4}: {5}
        - Pixeles con proba<{4}: {6}'''.format(i, output_model_file, output_proba_file, train_data_len, threshold, add_to_train_len , continue_pred_len))
        i += 1
        if (add_to_train_len == 0) or (continue_pred_len == 0):
            break

    # reemplaza na en columna pred_class con 'vc_original'
    # (los pixeles que no tiene pred_class son los pieles de verdad de campo originales)
    # y guarda la predicción final
    final_prediction = os.path.join(threshold_folder,f'randomforest_iterations_final_prediction.csv')
    iter_train_data.to_csv(final_prediction)
    final_deptos_prediction = os.path.join(threshold_folder,f'randomforest_iterations_deptos_prediction.csv')
    (
        iter_train_data
        .groupby(['cultivo','nombre'], as_index=False)
        .size()
        .rename(columns={'size':'pixels'})
        .assign(ha=lambda x: x.pixels*0.04)
        .to_csv(final_deptos_prediction, index=False)
    )

+++++ PREDICCIONES PARA THRESHOLD 0.3


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.9s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.2s finished



*** ITERACIÓN #000
        - Modelo guardado en ../model/threshhold_0.3/randomforest_iterations_000/model_000.joblib
        - Probabilidades guardadas en ../model/threshhold_0.3/randomforest_iterations_000/probas_000.npy
        - Pixeles de entrenamiento: 466
        - Pixeles con proba>=0.3: 1368
        - Pixeles con proba<0.3: 166


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.1s finished



*** ITERACIÓN #001
        - Modelo guardado en ../model/threshhold_0.3/randomforest_iterations_001/model_001.joblib
        - Probabilidades guardadas en ../model/threshhold_0.3/randomforest_iterations_001/probas_001.npy
        - Pixeles de entrenamiento: 1834
        - Pixeles con proba>=0.3: 166
        - Pixeles con proba<0.3: 0
+++++ PREDICCIONES PARA THRESHOLD 0.4


...


*** ITERACIÓN #010
        - Modelo guardado en ../model/threshhold_0.7/randomforest_iterations_010/model_010.joblib
        - Probabilidades guardadas en ../model/threshhold_0.7/randomforest_iterations_010/probas_010.npy
        - Pixeles de entrenamiento: 605
        - Pixeles con proba>=0.7: 1
        - Pixeles con proba<0.7: 1394


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.1s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.2s finished



*** ITERACIÓN #011
        - Modelo guardado en ../model/threshhold_0.7/randomforest_iterations_011/model_011.joblib
        - Probabilidades guardadas en ../model/threshhold_0.7/randomforest_iterations_011/probas_011.npy
        - Pixeles de entrenamiento: 606
        - Pixeles con proba>=0.7: 0
        - Pixeles con proba<0.7: 1394
+++++ PREDICCIONES PARA THRESHOLD 0.8


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    0.9s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.2s finished



*** ITERACIÓN #000
        - Modelo guardado en ../model/threshhold_0.8/randomforest_iterations_000/model_000.joblib
        - Probabilidades guardadas en ../model/threshhold_0.8/randomforest_iterations_000/probas_000.npy
        - Pixeles de entrenamiento: 466
        - Pixeles con proba>=0.8: 0
        - Pixeles con proba<0.8: 1534
